Upload some fake calendar data to Googles Dialogflow CX and ask it some questions

In [2]:
import gcp_fx

In [3]:
# uploaded this csv to gcs
gcp_fx.get_calendar().query(''' note==note ''')

downloading caltxt


,date_id,note
15,1/16/2025,wife bday
28,1/29/2025,son birthday
42,2/12/2025,anniversary
55,2/25/2025,hunting
56,2/26/2025,hunting
57,2/27/2025,hunting
58,2/28/2025,hunting
72,3/14/2025,fishing
73,3/15/2025,fishing
74,3/16/2025,fishing


In [ ]:
# then Agent Builder -> Create App -> Chat
# then attached a datastore, and import gcs document above to datastore    
from google.cloud import dialogflowcx_v3

def get_client_session():

    LOCATION = "us-central1"  
    api_endpoint = f"{LOCATION}-dialogflow.googleapis.com:443"
    client_options = {"api_endpoint": api_endpoint}
    client = dialogflowcx_v3.SessionsClient(client_options=client_options)

    AGENT_ID = "5e89b144-8110-454e-8c91-78ac825fddf9"  #
    # import uuid
    # SESSION_ID = str(uuid.uuid4())
    SESSION_ID = '99c15dbe-388e-4408-ae00-a31bce1b56f4'
    sessions_path = f"projects/{gcp_fx.PROJECT_ID}/locations/{LOCATION}/agents/{AGENT_ID}/sessions/{SESSION_ID}"
    return client, sessions_path

def submit_question(question):
    query_input = dialogflowcx_v3.QueryInput()
    query_input.text.text = question
    query_input.language_code = "en-us"

    client,sessions_path = get_client_session()
    request = dialogflowcx_v3.DetectIntentRequest(session=sessions_path,query_input=query_input)
    response = client.detect_intent(request=request)
    return response.query_result.response_messages[0].text

In [7]:
submit_question('hi ya')

text: "Hi! How are you doing?"

In [7]:
print(submit_question('when am i fishing'))

text: "You are fishing on the following dates:\n- 3/15/2025\n- 3/16/2025\n- 3/17/2025\n- 3/18/2025"



In [6]:
print(submit_question('when am i hunting'))

text: "You are hunting on the following dates:\n- 2/26/2025\n- 2/27/2025\n- 2/28/2025\n- 3/1/2025"



In [10]:
print(submit_question('when should i buy my wife a present'))

text: "Your wife\'s birthday is on 1/17/2025."



In [11]:
print(submit_question('what are notable days'))

text: "- Your wife\'s birthday is on 1/17/2025.\n- Your anniversary is on 2/13/2025.\n- Your son\'s birthday is on 1/30/2025."



In [12]:
print(submit_question('when am i free'))

text: "- You are free to go fishing from 3/15/2025 to 3/18/2025.\n- You are free to go hunting from 2/26/2025 to 3/1/2025.\n- You are free to take your dog to the park on 4/7/2025."

